In [1]:
# -*- coding: utf-8 -*-

###############################################
#                                             #
#     Created on Wed Jan 19 11:16:00 2022     #
#         @author: Chengpi Wu                 #
#                                             #
###############################################

# import stockDB as sd
# da = sd.StockDB()
# import FredData
# da.importFredData()

In [2]:

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


# import libraries coded before
import stockMatrix as sm
import stockDB as sd

import os

CURRENT_DIR = os. getcwd() 

pd.options.display.max_columns= 999
plt.rcParams['font.family'] = 'DeJavu Serif'
plt.rcParams['font.serif'] = ['Times New Roman']

import sys
###############################################
#                                             #
#   Step 1. Collect Data                      #
#                                             #
###############################################
import timeit

start = timeit.default_timer()

da = sd.StockDB()
da.checkN100Data()



ModuleNotFoundError: No module named 'yfinance'

In [ ]:
# da.loadStockPrice('QQQ')

In [ ]:
# da.loadStockPrice('TLT')

In [ ]:

symbolList=['DGS20','DGS30','M1SL','M2SL','QQQ','TLT']

dayData,actionData = da.prepareData(symbolList)


print(dayData.shape)
# print(dayData.head(5))
print(dayData.columns)


In [ ]:
dayData.insert(0, 'timeseries', range(0, len(dayData)))
print(dayData.head(5))

In [ ]:
print(dayData['Date'].min(),dayData['Date'].max())
dayData.dropna(axis = 0, inplace =True)
# print(dayData.tail(10))
print(dayData['Date'].min(),dayData['Date'].max())

In [ ]:

goalDays =[]
testData =[]
goalData =[]
for i in range(len(actionData)):
    print(actionData[i].shape)
    # print(goalData[i].head(3))
    goalData.append(actionData[i][(actionData[i]['Date'] >= '2003-01-01') & (actionData[i]['Date'] <= '2019-12-31') ]) # get the data segment
    testData.append(actionData[i][actionData[i]['Date'] > '2019-12-31' ].copy())  # get the data segment of test  
    print(goalData[i][['Date']].head(5))
    print(goalData[i].shape)
    print(testData[i].shape)

print(goalData[0].columns)
# goalData is the target data of training data 
# testData is the target data of testing data

In [ ]:

# ###############################################
# #                                             #
# #   Step 2. Prepare Data                      #
# #                                             #
# ###############################################

periodList = ['Week','Month','Season','Year']

dataset_X = []
dataset_Y = []
dataset_XY = []

testset_X = []
testset_Y = []
testset_XY = []

print(dayData.shape)

for i in range(0,4):
    print('#'*80)
    print(periodList[i] + ' Data')
    print(goalData[i].shape)
    # print(goalData[i].tail(10))
    print(dayData.shape)
    # print(goalData[i].tail(1))
    print(goalData[i]['Date'].min(),goalData[i]['Date'].max())
    # print(dayData.tail(1))
    print(dayData['Date'].min(),dayData['Date'].max())
    data_XY = pd.merge(goalData[i],dayData,on='Date',how='left') #.drop(['Date'], axis=1)
    data_XY.dropna(axis = 0, inplace =True)
    print(data_XY.shape)
    print('-'*80)
    print(data_XY.isna().values.any())
    print(data_XY.isna().count())
    
    print(data_XY[data_XY.isnull().T.any()])
    data_XY.dropna(axis = 0, inplace =True) # clear???
    print(data_XY.shape)
    # print(data_XY.tail(5))
    data_Y = data_XY[['TLT_buy_signal']]
    data_X = data_XY.drop(['Date','Day_QQQ','Day_TLT','TLT_buy_signal'], axis=1)
    data_XY.drop(['Date','Day_QQQ','Day_TLT'], axis=1,inplace =True)
    

    # data_Y = goalData[i].drop(['Date'], axis=1)
    # print(data_X.head(10))
    print(data_X.shape)
    print(data_Y.shape)   
    dataset_X.append(data_X)
    dataset_Y.append(data_Y)
    dataset_XY.append(data_XY)

    print('*'*80)
    print(dayData.shape)
    print(testData[i].shape)


    test_XY = pd.merge(testData[i],dayData,on='Date',how='left') #.drop(['Date'], axis=1)
    test_XY.dropna(axis = 0, inplace =True)
    test_Y = test_XY[['TLT_buy_signal']]
    test_X = test_XY.drop(['Date','Day_QQQ','Day_TLT','TLT_buy_signal'], axis=1)
    test_XY.drop(['Date','Day_QQQ','Day_TLT'], axis=1,inplace =True)
    print(test_X.shape)
    print(test_Y.shape)   
    testset_X.append(test_X)
    testset_Y.append(test_Y)
    testset_XY.append(test_XY)

# data_X,data_Y is the training data
# test_X,test_Y is the testing data




In [ ]:


###############################################
#                                             #
#   Step 3. Choose Model                      #
#                                             #
###############################################

import tensorflow as tf

# conda install tensorflow-estimator==2.4.1
tf.__version__
help(tf.estimator)


In [ ]:
import os
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
print('tensorflow version:', tf.__version__) # 2.4.1


In [ ]:

k = 0
# 0,1,2,3 --> week,month,season,year

data_X = dataset_X[k]
data_Y = dataset_Y[k]
data_XY = dataset_XY[k]

test_X = testset_X[k]
test_Y = testset_Y[k]
test_XY = testset_XY[k]

data_XY.dtypes


In [ ]:
df = data_XY.copy()
df.shape

In [ ]:
df.info(verbose=True)
df.dtypes

In [ ]:
len(df)

In [ ]:
pd.options.display.max_columns= 999
df.columns


In [ ]:
len(df)

In [ ]:
df.tail(2)             

In [ ]:
# df = np.asarray(df).astype('float32')
# df = pd.DataFrame(df)



In [ ]:
df[df.select_dtypes('float64').columns] = df.select_dtypes('float64').astype(np.float32)
df.head(2)

In [ ]:
target = df.pop('TLT_buy_signal')
target.head(2)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))


In [ ]:
for feat, targ in dataset.take(5):
    print ('Features: {}, Target: {}'.format(feat, targ))
  

train_dataset = dataset.shuffle(len(df)).batch(1)


In [ ]:
def get_compiled_model():
    model = tf.keras.Sequential([
      tf.keras.layers.Dense(10, activation='relu'),
      tf.keras.layers.Dense(10, activation='relu'),
      tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model


model = get_compiled_model()


In [ ]:

###############################################
#                                             #
#   Step 4. Train Model                       #
#                                             #
###############################################


In [ ]:

import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)

In [ ]:
history = model.fit(train_dataset, epochs=15)


In [ ]:
model.summary()


In [ ]:
history.history

In [ ]:
# print(test_X['Date'].min(),dayData['Date'].max())
# dayData.dropna(axis = 0, inplace =True)
# print(dayData.tail(10))
# print(test_X['Date'].min(),dayData['Date'].max())

In [ ]:
test_X.shape

In [ ]:
test_Y.shape

In [ ]:
test_Y.head(3)
# test_X.head(5)

In [ ]:
getData = pd.merge(test_X,dayData,on='timeseries',how='left')
# predicts = pd.merge(getData,test_Y,on='index',how='left')

predicts = getData.join(test_Y, how='left')
predicts[['Date','TLT_buy_signal']]

In [ ]:
###############################################
#                                             #
#   Step 5. Model Evaluation                  #
#                                             #
###############################################

print("Evaluate on test data")
results = model.evaluate(test_X, test_Y, batch_size=128) # 128
print("test loss, test acc:", results)

In [ ]:
# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for test data:")
predictions = model.predict(test_X)
print("Predictions shape:", predictions.shape)

In [ ]:
predictions.head(20)

In [ ]:
print('Training Data Period:')
print(goalData[0]['Date'].min(),goalData[0]['Date'].max())
# pd.DataFrame(goalData)[['Data']]

In [ ]:
print('Testing Data Period:')
print(testData[0]['Date'].min(),testData[0]['Date'].max())

In [ ]:

stop = timeit.default_timer()

print('Time(seconds): ', int(stop - start) )
